In [1]:
import sys
print(sys.version)
import numpy as np
import ROOT
from array import *
import matplotlib.pyplot as plt
import math
from scipy.optimize import brentq

3.12.3 (main, Apr 10 2024, 05:33:47) [GCC 13.2.0]


In [12]:
def resultsTOtxt(arr,  filename):
#arr is the concatenated and transposed array of all the arrays wished to be written
#arrDescription a string describing and is each collumn of the file
#writes out each array concatenated in arr as an collumn in the file
    file = open(filename, 'w')
    np.savetxt(filename, arr, delimiter=' ')
    file.close()

In [13]:
listsamples = ['m05_b1p1', 'm05_b1p5', 'm05_b10', 'm10_b1p1', 'm10_b1p5', 'm10_b10', 'm20_b1p1', 'm20_b1p5', 'm20_b10', 'm40_b1p1', 'm40_b1p5', 'm40_b10']
list_samples_latex = [r'$m_\chi = 5 \text{ GeV}, \; \gamma=1.1$', r'$m_\chi = 10 \text{ GeV}, \; \gamma=1.1$',r'$m_\chi = 20 \text{ GeV}, \; \gamma=1.1$', r'$m_\chi = 40 \text{ GeV}, \; \gamma=1.1$', r'$m_\chi = 5 \text{ GeV}, \; \gamma=1.5$', r'$m_\chi = 10 \text{ GeV}, \; \gamma=1.5$', r'$m_\chi = 20 \text{ GeV}, \; \gamma=1.5$', r'$m_\chi = 40 \text{ GeV}, \; \gamma=1.5$', r'$m_\chi = 5 \text{ GeV}, \; \gamma=10$', r'$m_\chi = 10 \text{ GeV}, \; \gamma=10$', r'$m_\chi = 20 \text{ GeV}, \; \gamma=10$', r'$m_\chi = 40 \text{ GeV}, \; \gamma=10$' ]
list_Nsimuevt_BDM = [100000,100000,100000,99800,99600,99200,99400,99800,95000,96000,97200,99000]

In [14]:
def CVNCut(CVN_CutProb,CVN_SignalReco,CVN_Bg,CosSun_SignalValidCVN,AtmNuCosSun):
    nSignalEvents = 0 
    nBGEvents = 0
    EventsSignalCut = CVN_SignalReco[CVN_SignalReco > CVN_CutProb]
    EventsBgCut = CVN_Bg[CVN_Bg > CVN_CutProb] 
    CosSunSignal_CVNCut = CosSun_SignalValidCVN[CVN_SignalReco > CVN_CutProb]
    CosSunBgCVNCut = AtmNuCosSun[CVN_Bg > CVN_CutProb]
    

    return CosSunSignal_CVNCut, CosSunBgCVNCut, EventsSignalCut, EventsBgCut

In [15]:

def BDTCut(BDT_Cut, BDT_Signal,BDT_Atm, CosSunSig_NoCVN, CosSunAtmNoCVN):
    BDT_SignalCut = BDT_Signal[BDT_Signal>BDT_Cut]
    BDT_AtmCut = BDT_Atm[BDT_Atm>BDT_Cut]
    CosSunSignal_BDTCut =  CosSunSig_NoCVN[BDT_Signal > BDT_Cut]
    CosSunAtm_BDTCut = CosSunAtmNoCVN[BDT_Atm > BDT_Cut]

    return CosSunSignal_BDTCut, CosSunAtm_BDTCut, BDT_SignalCut, BDT_AtmCut

In [16]:

def AngularCutBothSides(CVNScore_NC,BDTScore, nSignalSimulatedEvts): 
    
    results = []
    SprimeMin = 999999999
    bestCutRight = 9999
    bestCutLeft = 9999
    bestEff = 9999
    bestBkg = 9999
    bestBkgErr = 999
    Sprime = 999
    CosSunSignal = []
    CosSunAtm = []
    
    CosSunSignalBDTCut, CosSunAtmBDTCut, BDTSignalCut, BDTAtmCut = BDTCut(BDTScore, BDTResponseSignal,BDTResponseAtm, CosSun_Signal_NoCVN, CosSun_Atm_NoCVN)
    CosSunSignalCVNCut, CosSunBgCVNCut, SignalEvts, BgEvts = CVNCut(CVNScore_NC, CVN_Signal_Valid, CVN_Atm_Valid, CosSun_Signal_wCVN, CosSun_Atm_wCVN)
    
    nSimulatedEvtsAll_atm = 301000
    BackgroundScale = 4 * 10 * 0.732 *2495.98 /nSimulatedEvtsAll_atm
    # = 40kt * 10 years * reco/simulated ratio * # atm nu expected for 10kt y / simulated atm nu events 

    
    CosSunEvts = np.concatenate((CosSunSignalBDTCut,CosSunSignalCVNCut), axis=0)
    CosSunAtmEvts = np.concatenate((CosSunAtmBDTCut, CosSunBgCVNCut), axis=0)
    
    for CosCutRight in CosCutsToRight:
        for CosCutLeft in CosCutsToLeft:
            nReco = 0.0001
            nBG = 0.00001
            nRecoToTheRight = np.sum(CosCutRight < CosSunEvts)
            nRecoToTheLeft = np.sum(CosCutLeft > CosSunEvts)
            nReco = nRecoToTheRight + nRecoToTheLeft
            
            nBGToTheRight = np.sum(CosCutRight < CosSunAtmEvts)
            nBGToTheLeft = np.sum(CosCutLeft > CosSunAtmEvts)
            nBG = nBGToTheRight + nBGToTheLeft
            
            rReco = nReco/nSignalSimulatedEvts  # Selection Efficiency
            bEvents = nBG*BackgroundScale   # Number of background events
            

            #### FUNCTION TO FIND THE OPTIMAL SELECTION CUTS ####
            Sprime = 25./ (2.*rReco) + math.sqrt( 25.*bEvents/ (rReco*rReco) + 625./(4.*rReco*rReco) )
            
            if Sprime < SprimeMin:
                SprimeMin = Sprime
                bestCutRight = CosCutRight
                bestCutLeft = CosCutLeft
                bestEff = rReco
                bestBkg = bEvents
                bestBkgErr = math.sqrt( nBG *BackgroundScale)
                BestCVNCut = CVNScore_NC
                BestBDTCut = BDTScore
                CosSunSignal = CosSunEvts
                CosSunAtm = CosSunAtmEvts
            #### FUNCTION TO FIND THE OPTIMAL SELECTION CUTS ####


    results = [SprimeMin, bestCutLeft, bestCutRight, bestEff, bestBkg, bestBkgErr, CVNScore_NC, BDTScore, CosSunEvts, CosSunAtmEvts]
    return results



In [17]:
## Load Atmospheric Neutrinos Distributions

CVN_Atm = np.loadtxt('OptimalCuts/BDT_CVN_Ang_MaxCVN/CVN_Atm_hA_BR.txt', delimiter=' ') # Load CVN Atmospheric Neutrino Score 
BDTResponseAtm = np.loadtxt('OptimalCuts/BDT_CVN_Ang_MaxCVN/BDT_Atm_hA_BR.txt', delimiter=' ') # Load BDT Atmospheric Neutrino Score 
CosSun_Atm = np.loadtxt('OptimalCuts/BDT_CVN_Ang_MaxCVN/CosSun_Atm_hA_BR.txt', delimiter=' ') # Load Angular Distribution for  Atmospheric Neutrino w.r.t Sun 

CVN_Atm_Valid = CVN_Atm[CVN_Atm>-1] # Separate events with and without CVN Score
CosSun_Atm_wCVN = CosSun_Atm[CVN_Atm > -1]
CosSun_Atm_NoCVN = CosSun_Atm[CVN_Atm == -1]

Sensitivity_results = open("TestAngularSelections/Sensitivity_results_00a.txt","a")
Eff_Bkg_index =open("TestAngularSelections/Eff_Bkg_index_00a.txt","a")
Eff_Bkg_index.write("SigStrength\tNegCosCut\tPosCosCut\tbestEff\tBkgExpect\tBkgError\tCVNCut\tBDTCut\n")


73

In [18]:
for index, sig_tree in enumerate(listsamples):
    
    
    CVN_Signal = np.loadtxt('OptimalCuts/BDT_CVN_Ang_MaxCVN/CVN_'+sig_tree+'.txt', delimiter=' ') # Load CVN Boosted DM Score 
    CosSun_Signal = np.loadtxt('OptimalCuts/BDT_CVN_Ang_MaxCVN/CosSun_'+sig_tree+'.txt', delimiter=' ') # Load BDT Boosted DM Score 
    BDTResponseSignal = np.loadtxt('OptimalCuts/BDT_CVN_Ang_MaxCVN/BDT_'+sig_tree+'.txt', delimiter=' ') # Load Angular Distribution for Boosted DM w.r.t Sun 
    
    CVN_Signal_Valid = CVN_Signal[CVN_Signal>-1] # Separate events with and without CVN Score
    CosSun_Signal_wCVN = CosSun_Signal[CVN_Signal> -1]
    CosSun_Signal_NoCVN = CosSun_Signal[CVN_Signal == -1]

    ### Cuts Range to probe ###
    CosCutsToRight = np.linspace(0.0,1.00,10)
    CosCutsToLeft = np.linspace(0.0,-1.00,10)
    CVN_Cuts = np.linspace(0.5,1.00,10)
    BDT_Cuts =  np.linspace(-0.2,0.7,10)
    ### Cuts Range to probe ###
    
    bestResultsTwoAngCuts = []
    SpMinTwoAngCuts = 99999999.9
    bestCVNCutTwoCut = -3

    Sensitivity_results.write("Sample: "+sig_tree+"\n")
    Sensitivity_results.write("SigStrength\tNegCosCut\tPosCosCut\tbestEff\tBkgExpect\tBkgError\tCVNCut\tBDTCut\n")
    for CVNScore in CVN_Cuts:
        for BDT_Score in BDT_Cuts:
            resultsTwoAngCuts = AngularCutBothSides(CVNScore, BDT_Score, list_Nsimuevt_BDM[index])
            if resultsTwoAngCuts[0] < SpMinTwoAngCuts:
                bestResultsTwoAngCuts = resultsTwoAngCuts
                bestCVNCutTwoCut = CVNScore
                SpMinTwoAngCuts = resultsTwoAngCuts[0]
                Sensitivity_results.write(str(bestResultsTwoAngCuts[0])+"\t"+str(bestResultsTwoAngCuts[1])+"\t"+str(bestResultsTwoAngCuts[2])+"\t"+str(bestResultsTwoAngCuts[3])+"\t"+str(bestResultsTwoAngCuts[4])+"\t"+str(bestResultsTwoAngCuts[5])+"\t"+str(bestResultsTwoAngCuts[6])+"\t"+str(bestResultsTwoAngCuts[7])+"\n")
                

    Sensitivity_results.write(str(" ======= BEST CUTS ========")+"\n")
    Sensitivity_results.write(str(bestResultsTwoAngCuts[0])+"\t"+str(bestResultsTwoAngCuts[1])+"\t"+str(bestResultsTwoAngCuts[2])+"\t"+str(bestResultsTwoAngCuts[3])+"\t"+str(bestResultsTwoAngCuts[4])+"\t"+str(bestResultsTwoAngCuts[5])+"\t"+str(bestResultsTwoAngCuts[6])+"\t"+str(bestResultsTwoAngCuts[7])+"\n")   
    np.savetxt(Eff_Bkg_index, np.array(bestResultsTwoAngCuts[0:8]).flatten(), fmt="%.3f", newline = " ",delimiter=', ')
    Eff_Bkg_index.write("\n")
    Sensitivity_results.write("\n")
    print('\n')
    print(index)
    print("################# BEST ANGULAR AND CVN CUTS INFO #######################")
    print("Optimal Signal Strength: " + '{:.0f}'.format(bestResultsTwoAngCuts[0]))
    print("Optimal CVN Cut: " +'{:.3f}'.format(bestResultsTwoAngCuts[6]))
    print("Optimal BDT Cut: " +'{:.3f}'.format(bestResultsTwoAngCuts[7]))
    print("Optimal Positive Cossine Angular Cut: "  + '{:.3f}'.format(bestResultsTwoAngCuts[2]))
    print("Optimal Negative Cossine Angular Cut: " +  '{:.3f}'.format(bestResultsTwoAngCuts[1]))
    print("Signal Efficiency: " + '{:.3f}'.format(bestResultsTwoAngCuts[3]))
    print("")
    print("Number of expected Atm Nu events: " + '{:.1f}'.format(bestResultsTwoAngCuts[4]) + ", statistical uncertainty: " + '{:.1f}'.format(bestResultsTwoAngCuts[5]))
    print('\n')

    resultsTOtxt(bestResultsTwoAngCuts[8], 'TestAngularSelections/CosSunSignalAfterCuts_'+sig_tree+'.txt')
    resultsTOtxt(bestResultsTwoAngCuts[9], 'TestAngularSelections/CosSunAtmAfterCuts_'+sig_tree+'.txt')
   
Sensitivity_results.close()
Eff_Bkg_index.close()

/tmp/ipykernel_20458/1695104021.py:42: RuntimeWarning: divide by zero encountered in scalar divide
  Sprime = 25./ (2.*rReco) + math.sqrt( 25.*bEvents/ (rReco*rReco) + 625./(4.*rReco*rReco) )
/tmp/ipykernel_20458/1695104021.py:42: RuntimeWarning: invalid value encountered in scalar divide
  Sprime = 25./ (2.*rReco) + math.sqrt( 25.*bEvents/ (rReco*rReco) + 625./(4.*rReco*rReco) )




0
################# BEST ANGULAR AND CVN CUTS INFO #######################
Optimal Signal Strength: 4013
Optimal CVN Cut: 1.000
Optimal BDT Cut: 0.200
Optimal Positive Cossine Angular Cut: 0.222
Optimal Negative Cossine Angular Cut: -0.111
Signal Efficiency: 0.051

Number of expected Atm Nu events: 1449.3, statistical uncertainty: 38.1




1
################# BEST ANGULAR AND CVN CUTS INFO #######################
Optimal Signal Strength: 3588
Optimal CVN Cut: 1.000
Optimal BDT Cut: 0.100
Optimal Positive Cossine Angular Cut: 0.000
Optimal Negative Cossine Angular Cut: 0.000
Signal Efficiency: 0.064

Number of expected Atm Nu events: 1903.1, statistical uncertainty: 43.6




2
################# BEST ANGULAR AND CVN CUTS INFO #######################
Optimal Signal Strength: 3577
Optimal CVN Cut: 1.000
Optimal BDT Cut: 0.100
Optimal Positive Cossine Angular Cut: 0.000
Optimal Negative Cossine Angular Cut: 0.000
Signal Efficiency: 0.065

Number of expected Atm Nu events: 1903.1, statisti